# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [2]:
import pandas as pd
import json
import os
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

load_dotenv()

True

In [3]:
# Import N3 Vocabulary
file_path = '../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()

,expression,reading
0,題名,だいめい
1,女史,じょし
2,尻,しり
3,下,しも
4,ハイキング,ハイキング


#### load Models

In [4]:
# azure_llm = AzureChatOpenAI(
#     azure_endpoint="https://tooldev-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
#     api_key=os.environ["AZURE_API_KEY"],
#     model_name="gpt-4o",
#     api_version="2025-01-01-preview",
#     temperature=0.5,
# )

In [5]:
aws_llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
     # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs=dict(temperature=0.5),
    region = "us-east-2",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

## Question Explore

# Kanji 读假名（读音问题）

In [6]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    improved_question: str
    final_question: str


kanji_example = """
15. 大雪で朝から電車が（　）している。
	1.	縮小
	2.	滞在
	3.	延期
	4.	運休

16. 今日は暑かったので、シャツが（　）でぬれてしまった。
	1.	いびき
	2.	あくび
	3.	あせ
	4.	いき

17. 答えさんに声がよく聞こえるように、（　）を使って話してください。
	1.	サイレン
	2.	エンジン
	3.	ノック
	4.	マイク

18. 昨日は早く寝たが、夜中に大きな音がして目が（　）しまった。
	1.	嫌がって
	2.	覚めて
	3.	驚いて
	4.	怖がって

19. 林さんはいつも冗談ばかり言うので、その話も本当かどうか（　）。
	1.	あやしい
	2.	おそろしい
	3.	にくらしい
	4.	まずしい

20. 本日の面接の結果は、1 週間以内にメールで（　）します。
	1.	広告
	2.	合図
	3.	通知
	4.	伝言

21. 兄はいつも（　）シャツを着ているので、遠くにいてもすぐに見つかる。
	1.	派手な
	2.	盛んな
	3.	わがままな
	4.	身近な

22. ここに車を止めることは規則で（　）されていますから、すぐに移動してください。
	1.	支配
	2.	英殺
	3.	禁止
	4.	批判

23. このコートは古いがまだ着られるので、捨ててしまうのは（　）。
	1.	もったいない
	2.	しかたない
	3.	かわいらしい
	4.	こいかない

24. 弟への誕生日プレゼントは、誕生日まで弟に見つからないように、たんすの奥に（　）。
	1.	包んだ
	2.	隠した
	3.	囲んだ
	4.	閉じた

25. 山口さんは今度のパーティーには来られないかもしれないが、（　）誘うつもりだ。
	1.	十分
	2.	一応
	3.	けっこう
	4.	たいてい
"""

print(words)

# Nodes
def generate_question(state: QuestionState):
    """First LLM call to generate initial question"""

    msg = aws_llm.invoke(f"""You are a Japanese teacher. Your job is to write 11 vocabulary questions for candidates to identify the correct kanji writing of a given word in hiragana for a JLPT N3 level exam paper. The question format follows:
Each question presents a word in hiragana within a sentence, and candidates must choose the correct kanji representation from four options. The options should include one correct kanji form and three distractors that are plausible but incorrect. The JLPT exam paper includes a mix of easy, moderate, and difficult questions to accurately assess the test-taker’s proficiency across different aspects of the language.

"""
            f"The vocabulary should be restricted to N3 level, you can refer to the vocabulary in the word list, choosing random words for the questions"
            f"please refer the question examples following the formal exam paper"
            f"append the correct answer and explanation of main challenges and why teacher asks this question to candidate in chinese at each question"
            f"Finally, beautify markdown format"
            f"topic list: {state['topic']}"
            f"word list: {words}"
            f"formal exam paper: {kanji_example}")
    
    return {"question": msg.content}


# Build workflow
kanji_workflow = StateGraph(QuestionState)

# Add nodes
kanji_workflow.add_node("generate_question", generate_question)

# Add edges to connect nodes
kanji_workflow.add_edge(START, "generate_question")
kanji_workflow.add_edge("generate_question", END)

# Compile
kanji_graph = kanji_workflow.compile()


     expression reading
0            題名    だいめい
1            女史     じょし
2             尻      しり
3             下      しも
4         ハイキング   ハイキング
...         ...     ...
2134         通勤    つうきん
2135        命じる    めいじる
2136         切れ      きれ
2137         訓練    くんれん
2138         出版   しゅっぱん

[2139 rows x 2 columns]


In [7]:
# Show workflow
#display(Image(kanji_graph.get_graph().draw_png()))

In [8]:
# Invoke
kanji = kanji_graph.invoke({"topic": "店で商品の特徴や価格を尋ねる | 電車やバスの乗り方を尋ねる | 今日の天気や気温について話す| 映画やドラマの感想を話す | 仕事のスケジュールや業務内容を話す"})
display(Markdown(kanji["question"]))

# JLPT N3 Kanji Selection Questions

## 問題1: 店で商品の特徴や価格を尋ねる
この商品の（とくちょう）について教えてください。
1. 特微
2. 特徴
3. 特長
4. 特性

**正解: 2. 特徴**

**解释**: 这道题测试考生对"特徴"一词的正确汉字书写的掌握程度。"特徴"是N3级别的常见词汇，在购物场景中经常使用。干扰项中的"特微"、"特长"和"特性"都是形似的错误写法，考生需要识别正确的汉字组合。

## 問題2: 電車やバスの乗り方を尋ねる
この（えき）から新宿まで、どの電車に乗ればいいですか。
1. 駅
2. 役
3. 液
4. 訳

**正解: 1. 駅**

**解释**: 这道题考查基础交通词汇"駅"的正确书写。虽然这是较简单的N3词汇，但在询问交通方式时非常重要。干扰项"役"、"液"和"訳"的发音都与"えき"相似，测试考生是否能正确识别表示"车站"的汉字。

## 問題3: 今日の天気や気温について話す
今日の（てんき）予報によると、明日は雨が降るそうです。
1. 天気
2. 天期
3. 天記
4. 天机

**正解: 1. 天気**

**解释**: 这道题测试考生对"天気"这个基础词汇的掌握。虽然是较简单的词汇，但在日常谈论天气时必不可少。干扰项都是形似但不正确的组合，考查考生对正确汉字组合的识别能力。

## 問題4: 映画やドラマの感想を話す
この映画の（ないよう）はとても感動的でした。
1. 内用
2. 内容
3. 内陽
4. 内様

**正解: 2. 内容**

**解释**: 这道题考查"内容"一词的正确汉字写法，这是讨论电影或电视剧情节时常用的词汇。干扰项"内用"、"内陽"和"内様"都是看似合理但不正确的组合，测试考生是否能准确识别表示"内容"的正确汉字。

## 問題5: 仕事のスケジュールや業務内容を話す
明日の（かいぎ）は9時から始まります。
1. 会儀
2. 会議
3. 回議
4. 回儀

**正解: 2. 会議**

**解释**: 这道题测试考生对商务场景中常用词汇"会議"的掌握程度。干扰项中的"会儀"、"回議"和"回儀"都是形似但不正确的组合，考查考生对正确汉字组合的识别能力，特别是在工作环境中使用的词汇。

## 問題6: 店で商品の特徴や価格を尋ねる
この商品の（ねだん）はいくらですか。
1. 値段
2. 根段
3. 値断
4. 根断

**正解: 1. 値段**

**解释**: 这道题考查